<a href="https://colab.research.google.com/github/kavinraam/Rail-Index-Prediction-Model/blob/main/Model_x.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas scikit-learn xgboost lightgbm

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [ ]:
df = pd.read_csv("/content/drive/My Drive/datasets/training_data.csv")

In [ ]:
X = df[["SECCODE", "LINECODE", "KMFROM", "BLOCKNO", "PARAM", "RI1", "GMT"]]
y = df["RI2"]

In [ ]:
X = X.dropna()
y = y.loc[X.index]

In [ ]:
X = pd.get_dummies(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

In [ ]:
rf_params = {'n_estimators': [100, 200], 'max_depth': [None, 10, 20]}
xgb_params = {'n_estimators': [100, 200], 'max_depth': [3, 5]}
lgb_params = {'n_estimators': [100, 200], 'max_depth': [-1, 10]}

In [ ]:
rf_grid = GridSearchCV(RandomForestRegressor(random_state=42), rf_params, cv=3, scoring='r2', n_jobs=-1)
rf_grid.fit(X_train, y_train)

xgb_grid = GridSearchCV(XGBRegressor(random_state=42, verbosity=0), xgb_params, cv=3, scoring='r2', n_jobs=-1)
xgb_grid.fit(X_train, y_train)

lgb_grid = GridSearchCV(LGBMRegressor(random_state=42), lgb_params, cv=3, scoring='r2', n_jobs=-1)
lgb_grid.fit(X_train, y_train)


/usr/local/lib/python3.11/dist-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.016548 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 901
[LightGBM] [Info] Number of data points in the train set: 283049, number of used features: 6
[LightGBM] [Info] Start training from score 2.344881


GridSearchCV(cv=3, estimator=LGBMRegressor(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [-1, 10], 'n_estimators': [100, 200]},
             scoring='r2')

In [ ]:
def evaluate_model(name, model, X_test, y_test):
    preds = model.predict(X_test)
    print(f"\n{name} Performance:")
    print(f"R2 Score: {r2_score(y_test, preds):.4f}")
    print(f"MAE     : {mean_absolute_error(y_test, preds):.4f}")
    print(f"MSE     : {mean_squared_error(y_test, preds):.4f}")
    print(f"RMSE    : {np.sqrt(mean_squared_error(y_test, preds)):.4f}")


In [ ]:
evaluate_model("Random Forest", rf_grid.best_estimator_, X_test, y_test)
evaluate_model("XGBoost", xgb_grid.best_estimator_, X_test, y_test)
evaluate_model("LightGBM", lgb_grid.best_estimator_, X_test, y_test)


Random Forest Performance:
R2 Score: 0.5427
MAE     : 0.0916
MSE     : 0.0256
RMSE    : 0.1600

XGBoost Performance:
R2 Score: 0.4931
MAE     : 0.1024
MSE     : 0.0284
RMSE    : 0.1684

LightGBM Performance:
R2 Score: 0.4684
MAE     : 0.1063
MSE     : 0.0297
RMSE    : 0.1725
